In [1]:
from SPARQLWrapper import SPARQLWrapper, JSON

sparql = SPARQLWrapper('https://query.wikidata.org/sparql')
sparql.setQuery('''
SELECT ?article WHERE {
    ?article schema:isPartOf <https://vi.wikipedia.org/>.

    SERVICE wikibase:label {
       bd:serviceParam wikibase:language 'vi'
    }
}
''')
sparql.setReturnFormat(JSON)
results = sparql.query().convert()
articals = results['results']['bindings']
urls = [artical['article']['value'] for artical in articals]
len(urls)

1644130

In [3]:
import pickle

with open('urls.pkl', 'wb') as f:
    pickle.dump(urls, f)

In [4]:
import pickle

with open('urls.pkl', 'rb') as f:
    urls = pickle.load(f)

In [9]:
from bs4 import BeautifulSoup
import requests
from tqdm import tqdm
import os

for idx, url in tqdm(enumerate(urls)):
    if idx < 20488:
        continue
    r = requests.get(url)
    html = BeautifulSoup(r.text, 'html.parser')

    title = html.select('#firstHeading')[0].text
    paragraphs = html.select('p')

    contents = [para.text.strip('\n').strip('\t').strip(' ') for para in paragraphs]
    try:
        contents.remove('Trang dành cho người dùng chưa đăng nhập tìm hiểu thêm')
    except:
        pass

    intro = 'Bài viết: ' + title + '\nĐường dẫn bài viết trên wikipedia: ' + urls[14] + '\n###\n' + '\n'.join(contents)

    sec = str(idx // 1000).zfill(5)
    if not os.path.exists(os.path.join('data', f'data{sec}')):
        os.makedirs(os.path.join('data', f'data{sec}'))

    with open(os.path.join('data', f'data{sec}', f'{idx}.txt'), 'w', encoding = 'utf-8') as f:
        f.write(intro)

28477it [2:16:23,  1.18s/it] 

In [43]:
0 // 100

0